In [1]:
%cd ..

/home/ltorres/leo/Cloud-Classification/code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(7)
np.random.seed(7)

In [5]:
import random

In [6]:
import os
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import networkx as nx
import glob

In [7]:
from src import config
from src import engine
from src import utils

In [8]:
from recipe.utils import *

In [9]:
torch.manual_seed(7)
np.random.seed(7)

### Dataloaders

In [10]:
def get_gcd_paths(data_dir, dataset_type):
    return glob.glob(
                     os.path.join(data_dir,f'GCD/{dataset_type}/**/*.jpg'), 
                     recursive=True
                    )

In [11]:
path_train_images = get_gcd_paths('/data/mandonaire','train')
#random.shuffle(path_train_images)


train_dataset = GCD(path_train_images, resize=160,augmentation=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=8,
        num_workers=8,
        shuffle=True,
    )

In [12]:
len(train_dataset)

10000

In [13]:
path_test_images = get_gcd_paths('/data/mandonaire','test')

test_dataset = GCD(path_test_images, resize=160, augmentation=False)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=8,
        num_workers=8,
        shuffle=True,
    )

In [14]:
len(test_dataset)

9000

## Model

In [15]:
from ViG.src.models import IsotropicVIG

In [16]:
model = IsotropicVIG(img_hw=160, in_channels=64, out_channels=7, num_blocks=2, kernel_size=8, patch_size=8)

In [17]:
from torchsummary import summary

In [18]:
device = 'cuda:3'
EPOCHS = 200
LR = 3e-4

### Logger

In [19]:
import wandb

In [20]:
wandb.init(
    # Set the project where this run will be logged
    project="cloud classification - VIG",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name="Isotropic ViG - S2", 
    # Track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "architecture": "Isotropic ViG - S2",
    "loss": "CrossEntropy",
    "optim": "SGD",
    "dataset": "GCD",
    "epochs": EPOCHS,
  })

wandb: Currently logged in as: leonardtd (use `wandb login --relogin` to force relogin)


### Training

In [21]:
model = model.to(device)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)



best_accuracy=0


for e in range(EPOCHS):
    train_predictions, train_targets, train_loss = train(model, train_loader, criterion, optimizer, device)
    test_predictions, test_targets, test_loss = test(model, test_loader, criterion, device)
    
    scheduler.step()
    
    train_acc = accuracy_score(train_targets, train_predictions)
    test_acc = accuracy_score(test_targets, test_predictions)
    

    print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format( 
        e+1,
        train_acc,
        train_loss,
        test_acc,
        test_loss
    ))
    
    
    if test_acc > best_accuracy:
        torch.save(model.state_dict(), "./ViG/vig_parameters.pt")
        print(f"Saved best parameters at epoch {e+1}")
        best_accuracy = test_acc
        
    
    metrics = {
                "train/train_loss": train_loss,
                "train/train_accuracy": train_acc,
                "test/test_loss": test_loss,
                "test/test_accuracy": test_acc,
              }

    wandb.log(metrics)    

100%|██████████| 1125/1125 [00:08<00:00, 125.28it/s]


EPOCH 1: Train acc: 48.50% Train Loss: 1.8541 Test acc: 40.53% Test Loss: 2.8742
Saved best parameters at epoch 1


100%|██████████| 1125/1125 [00:09<00:00, 124.55it/s]


EPOCH 2: Train acc: 59.64% Train Loss: 1.1538 Test acc: 52.26% Test Loss: 2.5859
Saved best parameters at epoch 2


100%|██████████| 1125/1125 [00:08<00:00, 125.32it/s]


...

EPOCH 198: Train acc: 83.63% Train Loss: 0.4125 Test acc: 65.96% Test Loss: 2.2896


100%|██████████| 1125/1125 [00:08<00:00, 125.08it/s]


EPOCH 199: Train acc: 83.95% Train Loss: 0.4138 Test acc: 67.11% Test Loss: 2.1777


100%|██████████| 1125/1125 [00:09<00:00, 124.55it/s]

EPOCH 200: Train acc: 83.94% Train Loss: 0.4146 Test acc: 66.76% Test Loss: 2.3019


In [24]:
wandb.finish()

test/test_accuracy,▁▄▅▄▅▅▆▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇█▇████▇█▇██▇███
test/test_loss,█▅▁▄▂▃▃▂▂▃▅▄▃▇▆▅▅▄▄▃▄▅▅▅▇▃▅▄▃▄▃▅▄▄▄▃▅▄▃▅
train/train_accuracy,▁▄▅▅▆▆▆▇▇▇▇█████████████████████████████
train/train_loss,█▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test/test_accuracy,0.66756
test/test_loss,2.30195
train/train_accuracy,0.8394
train/train_loss,0.41463
